In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
DATASET_DIR = './dataset/'

train_generator = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=30,
    width_shift_range=0.25,
    height_shift_range=0.25,
    brightness_range=(0.25, 1.25),
    zoom_range=0.25
)

test_generator = ImageDataGenerator(
    rescale=1./255
)

train = train_generator.flow_from_directory('./dataset/Train', target_size=(128, 128), class_mode='categorical', batch_size=512)
test = test_generator.flow_from_directory('./dataset/Test', target_size=(128, 128), class_mode='categorical', batch_size=1, shuffle=False)

In [ ]:
model = Sequential()

model.add(Conv2D(16, kernel_size=(3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(Conv2D(16, kernel_size=(3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(43, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [ ]:
model.summary()

In [ ]:
history = model.fit(train, validation_data=test, epochs=20, shuffle=True)

In [ ]:
#history = history.history
plt.figure()
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.grid()
plt.legend(['loss', 'val_loss'])
plt.show()

plt.figure()
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.grid()
plt.legend(['accuracy', 'val_accuracy'])
plt.show()

In [ ]:
model.evaluate(test)

In [ ]:
model = load_model('model_augment.h5')

In [ ]:
pred_y = model.predict(test)
pred_y = np.argmax(pred_y, axis=1)

In [ ]:
def display_confusion_matrix(cmat):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Blues')
    ax.set_xticks(range(43))
    ax.set_xticklabels(range(43), fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(43))
    ax.set_yticklabels(range(43), fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""

    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_true=test.classes, y_pred=pred_y)
display_confusion_matrix(confusion_matrix)